In [2]:
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score

In [3]:
sys.path.insert(0, '/home/reflex/refleX/lib/fastai')
from fastai.conv_learner import *


In [4]:
sys.path.insert(0, '/home/reflex/refleX/src')
from custom_metrics import f2, average_precision, average_recall, hamming_score


In [5]:
def get_data (model, PATHS, img_size, custom_transformations, batch_size, val_idxs):
    tfms = tfms_from_model(model, img_size, aug_tfms=custom_transformations, max_zoom=1.1)
    return ImageClassifierData.from_csv(PATHS['DATA_PATH'], PATHS['DATA_FOLDER'], PATHS['CSV_PATH'], tfms=tfms,
                    suffix='.png', val_idxs=val_idxs, bs=batch_size, num_workers=2)

In [6]:
PATHS = {
    'CSV_PATH': '/home/reflex/refleX/metadata/csv/fastai_train_test_split20_reflex.csv',
    'DATA_PATH': '/home/reflex/refleX/metadata/labeled/',
    'DATA_FOLDER': 'original512',
    'EXP_PATH': '/home/reflex/refleX/results/fastai_experiments'
}

In [7]:
val_idxs = list(range(0, 2227)) # 2228 -> ERROR -> ? #(1782, 2227)

In [8]:
# Definiujemy obiekt learner'a
model = resnet34
model_weights = '/home/reflex/refleX/results/fastai_experiments/first_search_overview/models/best_resnet34_imgsize512_batch8_unfreeze'
data = get_data(model, PATHS, 512, None, 8, val_idxs)
# for cross validation see https://forums.fast.ai/t/cross-validation-with-fast-ai/7988
learn = ConvLearner.pretrained(model, data, ps=0.5, metrics=[f2, average_precision, average_recall, hamming_score])
learn.load(model_weights)
data.classes

['artifact',
 'background_ring',
 'diffuse_scattering',
 'ice_ring',
 'loop_scattering',
 'non_uniform_detector',
 'strong_background']

In [9]:
learn

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [10]:
trimmed_network = nn.Sequential(*learn[:-2])

In [14]:
pretrained_model = resnet18(pretrained=True)

my_model1 = nn.Sequential(*list(pretrained_model.children())[:-1])
my_model2 = nn.Sequential(*list(pretrained_model.children())[:-1])

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.feature1 = my_model1
        self.feature2 = my_model2
        self.fc = nn.Linear(1024, 117)
    
    def forward(self, x):
        x1 = self.feature1(x)
        x2 = self.feature2(x)
        x3 = torch.cat((x1, x2), 1)
        x3 = x3.view(x3.size(0), -1)
        x3 = self.fc(x3)
        return x3

In [15]:
resnet18

<function torchvision.models.resnet.resnet18(pretrained=False, **kwargs)>

In [17]:
net = Net()
list(net.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
   (2): ReLU(inplace)
   (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
   (4): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
       (relu): ReLU(inplace)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
       (relu): ReLU(inplace)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      

In [18]:
learn = ConvLearner.pretrained(net, data, ps=0.5, metrics=[f2, average_precision, average_recall, hamming_score])

AttributeError: 'bool' object has no attribute 'dim'